In [1]:
import os
import sys
sys.path.append('../../src')
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import time
from hydra import initialize, compose
import pathlib
from sklearn.model_selection import train_test_split
# import config

import data_process.neg_sample as ng_sample
from data_process.utils import mix_merge
from data_process.data_split import data_split_user
from evaluate_entity import CustomHR, CustomNDCG, CustomRoc, CustomRoctop, CustomRecall_top, CustomPrecision_top
from model_entity import EntityCat_num
from data_utils import CatData
from utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL


from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score, precision_score

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# import argparse
torch.manual_seed(0)

In [2]:
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator, RemovableEventHandle
from ignite.metrics import Accuracy, Loss, Metric
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.exceptions import NotComputableError
from ignite.metrics.metric import sync_all_reduce, reinit__is_reduced
from ignite.contrib.handlers.tqdm_logger import ProgressBar
# from ignite.contrib.handlers import TensorboardLogger 
from ignite.contrib.handlers.wandb_logger import *

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config", overrides=[])

In [5]:
if device.type =='cpu':
    BATCH_SIZE = cfg.params.batch_size_cpu
    EPOCHS  = cfg.params.epochs_cpu
else:
    BATCH_SIZE = cfg.params.batch_size_gpu
    EPOCHS  = cfg.params.epochs_gpu

In [6]:
if device.type == 'cpu':
    use_amp=False
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test)).iloc[:202,]
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).iloc[:100,].reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).iloc[:100*cfg.params.neg_train,].reset_index(drop=True)
else:
    use_amp=True
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test))
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)

In [7]:
df_train_pos[DEFAULT_RATING_COL] = 1

In [8]:
def concat_index(df1, df2):
    df2.index = df2.index//cfg.params.neg_train
    return pd.concat([df1, df2], axis=0).sort_index(kind='mregesort').reset_index(drop=True)

In [9]:
df_train_all = concat_index(df_train_pos, df_train_neg)

In [10]:
df_train_all['flag'] = 1
df_test_ori['flag'] = 0
df_all = pd.concat([df_train_all, df_test_ori], axis=0).reset_index(drop=True)

user features: 
       'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany',
       
job features: 
       'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate',

# Adding the Sbert embedding

In [11]:
job_title_emb = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.title_emb))

In [12]:
job_title_emb.rename(columns={'item_id':DEFAULT_ITEM_COL, 'Title_em':'title_em'}, inplace=True)

In [36]:
title_em = pd.merge(df_all, job_title_emb[['title_em', 'itemid']], how='left', on=[DEFAULT_ITEM_COL]).dropna()

In [40]:
df_all = title_em.iloc[:,:-1]

In [42]:
np_title_em = np.reshape(np.concatenate(title_em.title_em.values).ravel(), (-1,768))


MemoryError: Unable to allocate 29.2 GiB for an array with shape (7848906240,) and data type float32

### Choose the features and process data for the training

In [18]:
user_features = ['City']
user_features_extend = [DEFAULT_USER_COL] + user_features

item_features = ['City_job']
item_features_extend =[DEFAULT_ITEM_COL] + item_features

base_features = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL]

In [19]:
df_mix_merge = mix_merge(df_all , df_all_features, user_features_extend, item_features_extend)

In [20]:
df_mix_merge.columns

Index(['userid', 'City', 'itemid', 'City_job', 'rating'], dtype='object')

In [21]:
def _cat_encode(df_data, list_f, encoder):
    for f in list_f:
        df_data[f] = encoder.fit_transform(df_data[f].astype('category').cat.codes.values)
    return df_data

In [22]:
def _embedding_dimension(df_all_encode, features_to_train, max_dim=50):

    embedding_size = []
    features_to_em = [i for i in features_to_train if i !=DEFAULT_RATING_COL]
    for c in features_to_em:
        num_unique_values = int(df_all_encode[c].nunique())
        embed_dim = int(min(np.ceil(num_unique_values/2), max_dim))
        embedding_size.append([num_unique_values, embed_dim])  
    return embedding_size

In [23]:
def encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=50):
    encoder = preprocessing.LabelEncoder()
    df_all_encode = _cat_encode(df_mix_merge, features_to_code, encoder)
    df_train = df_all_encode[df_all.flag==1]
    df_test = df_all_encode[df_all.flag==0]
    df_train = df_train[features_to_train]
    df_test = df_test[features_to_train]
    embedding_size = _embedding_dimension(df_all_encode, features_to_train, max_dim)
    return df_train, df_test, embedding_size

In [24]:
num_feature=[]
features_to_code = df_mix_merge.columns
features_to_train = [DEFAULT_USER_COL, DEFAULT_ITEM_COL]+ user_features + item_features+[DEFAULT_RATING_COL]


In [25]:

df_train,  df_test, embedding_size = encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=cfg.params.emb_dim)

print(f'The size of embedding layers:{embedding_size}')

The size of embedding layers:[[36, 18], [413, 50], [35, 18], [230, 50]]


In [26]:
features_to_train = [DEFAULT_USER_COL, DEFAULT_ITEM_COL]+ user_features + item_features+[DEFAULT_RATING_COL]
df_train= df_train[features_to_train]
df_test= df_test[features_to_train]

In [27]:
len_train = df_train.shape[0]
len_test = df_test.shape[0]

In [28]:
df_train_cat = np.concatenate((df_train.values[:, 0:-1], np_title_em[:len_train,]), axis=1)

np_train_cat = np.concatenate((df_train_cat, df_train.values[:, -1][...,None]), axis=1)

In [29]:
df_test_cat = np.concatenate((df_test.values[:, 0:-1], np_title_em[len_train:,]), axis=1)

np_test = np.concatenate((df_test_cat, df_test.values[:, -1][...,None]), axis=1)

In [30]:
np_train, np_val = train_test_split(np_train_cat, test_size=0.2)

In [31]:
# df_train_split, df_val_split = data_split_user(df_train, val_size=0.2)

# np_train = df_train_split.values
# np_val = df_val_split.values


In [32]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [33]:
train_dataset = CatData(np_train)
val_dataset = CatData(np_val)
test_dataset = CatData(np_test) 
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
val_loader = data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
test_loader = data.DataLoader(test_dataset, batch_size=cfg.params.neg_test+1, shuffle=False, num_workers=0,worker_init_fn=seed_worker,generator=g )

In [34]:
model = EntityCat_num(embedding_size = embedding_size, num_numerical_cols = 768,
               output_size = 2)
model.to(device)

EntityCat_num(
  (all_embeddings): ModuleList(
    (0): Embedding(36, 18)
    (1): Embedding(413, 50)
    (2): Embedding(35, 18)
    (3): Embedding(230, 50)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=904, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=100, out_features=2, bias=True)
  )
)

In [35]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=cfg.params.lr)

In [36]:
print(model)

EntityCat_num(
  (all_embeddings): ModuleList(
    (0): Embedding(36, 18)
    (1): Embedding(413, 50)
    (2): Embedding(35, 18)
    (3): Embedding(230, 50)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=904, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=100, out_features=2, bias=True)
  )
)


## Model the training 

In [37]:
def output_trans_loss(output):
    return output['y_pred'], output['label']

val_metrics_train = {
    'auc': CustomRoc(),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

val_metrics_test = {
    'hr': CustomHR(),
    'ndcg': CustomNDCG(),
    'auc': CustomRoc(),
    'roc_top': CustomRoctop(),
    'recall_top': CustomRecall_top(threshold=0.5),
    'precision_top': CustomPrecision_top(threshold=0.5),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

In [38]:
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    with torch.cuda.amp.autocast(enabled=use_amp):
        y_pred = model(x)[:,1]
        loss = criterion(y_pred, y.float())
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    return loss.item()

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)[:,1]
        label=label.float()
        return {'label':label, 'y_pred':y_pred}

def test_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)[:,1]
        label=label.float()
        y_pred_top, indices = torch.topk(y_pred, engine.state.topk)
        y_pred_top = y_pred_top.detach().cpu().numpy()
        reco_item = torch.take(x[:,1], indices).cpu().numpy().tolist()
        pos_item = x[0,1].cpu().numpy().tolist()  # ground truth, item id
        label_top = label[indices].cpu().numpy()
        return {'pos_item':pos_item, 'reco_item':reco_item, 'y_pred_top':y_pred_top, 'label_top':label_top, 'label':label, 'y_pred':y_pred}

trainer = Engine(train_step)

train_evaluator = Engine(validation_step)
# train_evaluator.state_dict_user_keys.append('topk')

val_evaluator = Engine(validation_step)
# val_evaluator.state_dict_user_keys.append('topk')

test_evaluator = Engine(test_step)
test_evaluator.state_dict_user_keys.append('topk')

# @val_evaluator.on(Events.STARTED)
# def init_user_value():
#     val_evaluator.state.topk=3
    
# @train_evaluator.on(Events.STARTED)
# def init_user_value():
#     train_evaluator.state.topk=3

@train_evaluator.on(Events.STARTED)
def init_user_value():
    test_evaluator.state.topk=cfg.params.topk
    
    
# Attach metrics to the evaluators
for name, metric in val_metrics_train.items():
    metric.attach(train_evaluator, name)

for name, metric in val_metrics_train.items():
    metric.attach(val_evaluator, name)

    
for name, metric in val_metrics_test.items():
    metric.attach(test_evaluator, name)
    
# Eearly_stop 
def score_function(engine):
    val_loss = engine.state.metrics['auc']
    return val_loss

Eearly_stop_handler = EarlyStopping(patience=cfg.params.patience, score_function=score_function, trainer=trainer)
val_evaluator.add_event_handler(Events.COMPLETED, Eearly_stop_handler)

In [39]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Training Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Validation Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')

    
@trainer.on(Events.COMPLETED)
def log_test_results(trainer):
    test_evaluator.run(test_loader)
    metrics = test_evaluator.state.metrics
    hr = metrics['hr']
    ndcg = metrics['ndcg']
    auc = metrics['auc']
    roc_top = metrics['roc_top']
    recall = metrics['recall_top']
    precision = metrics['precision_top']
    loss = metrics['loss']
    print(f"Test Results - Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
     Avg ndcg: {ndcg:.2f}  Avg auc: {auc:.2f}  Avg auc_top: {roc_top:.2f} \
      Avg recall: {recall:.2f}  Avg precision: {precision:.2f}")

pbar = ProgressBar(persist=False)
# pbar.attach(trainer)
pbar.attach(trainer)
# from ignite.handlers import Timer
# timer = Timer(average=True)
# timer.attach(trainer,
#              start=Events.STARTED,
#              resume=Events.EPOCH_STARTED,
#              pause=Events.EPOCH_COMPLETED,
#              step=Events.EPOCH_COMPLETED)

In [40]:
np_train.shape

(400, 773)

In [41]:
trainer.run(train_loader, max_epochs=EPOCHS)

/Users/hao/.pyenv/versions/3.6.15/envs/torch-cpu5/lib/python3.6/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/hao/.pyenv/versions/3.6.15/envs/torch-cpu5/lib/python3.6/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Training Results- Epoch[1]  Avg loss: 0.82           Avg auc:0.78
Validation Results- Epoch[1]  Avg loss: 0.78           Avg auc:0.83


/Users/hao/.pyenv/versions/3.6.15/envs/torch-cpu5/lib/python3.6/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Training Results- Epoch[2]  Avg loss: 0.76           Avg auc:0.80
Validation Results- Epoch[2]  Avg loss: 0.74           Avg auc:0.81


Training Results- Epoch[3]  Avg loss: 0.72           Avg auc:0.82
Validation Results- Epoch[3]  Avg loss: 0.71           Avg auc:0.82


...

Training Results- Epoch[19]  Avg loss: 0.65           Avg auc:0.94
Validation Results- Epoch[19]  Avg loss: 0.67           Avg auc:0.87


Epoch [20/20]: [19/20]  95%|█████████▌ [00:00<00:00]

Training Results- Epoch[20]  Avg loss: 0.65           Avg auc:0.94


Validation Results- Epoch[20]  Avg loss: 0.67           Avg auc:0.87
Test Results - Epoch[20]  Avg loss: 0.81      Avg ndcg: 0.00  Avg auc: 0.39  Avg auc_top: 0.00       Avg recall: 0.00  Avg precision: 0.00


State:
	iteration: 400
	epoch: 20
	epoch_length: 20
	max_epochs: 20
	output: 0.6799803972244263
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: torch.utils.data.dataloader.DataLoader
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [42]:
np_test.shape

(202, 773)

In [43]:
np_train.shape

(400, 773)

In [44]:
np.array([1,2,3,4,5])[3:]

array([4, 5])

In [45]:
np.array([1,2,3,4,5])[:3]

array([1, 2, 3])

In [46]:
config_dict = dict(cfg.params)
config_dict['Features']='-'.join(user_features+item_features)
wandb_logger = WandBLogger(
    project="pytorch-jrs",
    name="-".join(user_features)+'-'+'-'.join(item_features),
    config=config_dict,
    tags=["entity", "jrs"]
)

to_save = {'model': model}
checkpoint_handler = ModelCheckpoint(
    wandb_logger.run.dir,
    n_saved=2, filename_prefix='best',
    score_name="auc",
    global_step_transform=global_step_from_engine(trainer)
)

val_evaluator.add_event_handler(Events.COMPLETED, checkpoint_handler, to_save)

    
wandb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED,
    tag="training",
    output_transform=lambda loss: {"loss": loss}
)

wandb_logger.attach_output_handler(
    train_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="training",
    metric_names=['loss','auc'],
    global_step_transform=lambda *_: trainer.state.iteration,
)

wandb_logger.attach_output_handler(
    val_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="validation",
    metric_names=['loss',"auc"],
    global_step_transform=lambda *_: trainer.state.iteration,
)


wandb_logger.attach_output_handler(
    test_evaluator,
    event_name=Events.COMPLETED,
    tag="test",
    metric_names=['loss',"auc", 'hr', 'ndcg', 'roc_top', 'recall_top', 'precision_top'],
    global_step_transform=lambda *_: trainer.state.iteration,
)


wandb_logger.attach_opt_params_handler(
    trainer,
    event_name=Events.ITERATION_STARTED,
    optimizer=optimizer,
    param_name='lr'  # optional
)

# wandb_logger.watch(model) 

# trainer.run(train_loader, max_epochs=EPOCHS)
wandb_logger.close()

wandb: Currently logged in as: tiyuok2023. Use `wandb login --relogin` to force relogin
